In [6]:
import os
import sys

# Dynamically add the src directory to the path
sys.path.append(os.path.abspath("../src"))

In [26]:
import pandas as pd
import polars as pl

SOURCE = "../data/sampled_df.csv"
COLS = [
    "Year",
    "DEDonorcode",
    "DonorName",
    "DERecipientcode",
    "RecipientName",
    "FlowCode",
    "FlowName",
    # "Bi_Multi",
    # "Category",
    # "Finance_t",
    # "Aid_t",
    "USD_Commitment",
    "USD_Disbursement",
    "USD_Received",
    "USD_Commitment_Defl",
    "USD_Disbursement_Defl",
    "USD_Received_Defl",
    # "CurrencyCode",
    # "Commitment_National",
    # "Disbursement_National",
    # "USD_GrantEquiv",
    # "Geography",
    # "LDCflag",
    # "LDCflagName",
    #  'SDGfocus',
    #  'Keywords',
    #  'Gender',
    #  'Environment',
    #  'DIG',
    #  'Trade',
    #  'RMNCH',
    #  'DRR',
    #  'Nutrition',
    #  'Disability',
    #  'FTC',
    #  'PBA',
    #  'InvestmentProject',
    #  'AssocFinance',
    "Biodiversity",
    "ClimateMitigation",
    "ClimateAdaptation",
    "Desertification",
    "climate_relevance",
    "climate_class_number",
    "climate_class",
    "meta_category",
    "labelled_bilateral",
    "DonorType",
]

### pandas

In [27]:
df = pd.read_csv(SOURCE, usecols=COLS)

### polars

In [28]:
df = pl.read_csv(SOURCE, columns=COLS)

In [30]:
from typing import Literal


def read_data(
    selected_type: Literal["donors", "recipients"],
    source: str,
    columns: list,
    donor_type: Literal["bilateral", "multilateral", "all"],
) -> pl.DataFrame:
    """Read the data from the source and return the data based on the selected type and donor type.

    Args:
        selected_type (Literal["donors", "recipients"]): Whether to return the donors or recipients
        source (str): The path to the source file
        columns (list): The columns to read from the source
        donor_type (Literal["bilateral", "multilateral", "all"]): The type of donor to filter the data

    Returns:
        pl.DataFrame: The data based on the selected type and donor type
    """
    df = pl.read_csv(source=source, columns=columns)

    data = reshape_by_type(df, selected_type)

    data = filter_data_by_donor_type(data, donor_type)

    return data


def reshape_by_type(df: pl.DataFrame, selected_type: str) -> pl.DataFrame:
    """Reshape the table based on the selected type."""
    if selected_type == "donors":
        return df.drop(["DERecipientcode", "RecipientName"]).rename(
            {"DEDonorcode": "DonorCode"}
        )
    elif selected_type == "recipients":
        return df.drop(["DEDonorcode", "DonorName"]).rename(
            {"DERecipientcode": "RecipientCode"}
        )
    else:
        raise ValueError(
            "Invalid selected type. Please select either 'donors' or 'recipients'."
        )


def filter_data_by_donor_type(df: pl.DataFrame, donor_type: str) -> pl.DataFrame:
    """Filter the data based on the donor type."""
    if donor_type in ["bilateral", "multilateral"]:
        return df.filter(
            df["DonorType"]
            == ("Donor Country" if donor_type == "bilateral" else "Multilateral Donor")
        )
    return df  # No filtering needed if donor_type is "all"


read_data("donors", source=SOURCE, columns=COLS, donor_type="all")

Year,DonorCode,DonorName,FlowCode,FlowName,USD_Commitment,USD_Disbursement,USD_Received,USD_Commitment_Defl,USD_Disbursement_Defl,USD_Received_Defl,Biodiversity,ClimateMitigation,ClimateAdaptation,Desertification,climate_relevance,climate_class_number,climate_class,meta_category,labelled_bilateral,DonorType
i64,str,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,i64,str
2018,"""SWE""","""Sweden""",11,"""ODA Grants""",null,0.039124,null,null,0.038253,null,0.0,0.0,0.0,0.0,0.0,500.0,"""500""",null,1,"""Donor Country"""
2017,"""FRA""","""France""",11,"""ODA Grants""",0.022545,0.022545,null,0.023103,0.023103,null,1.0,0.0,1.0,0.0,0.0,500.0,"""500""",null,1,"""Donor Country"""
2022,"""USA""","""United States""",11,"""ODA Grants""",null,0.000603,null,null,0.000603,null,0.0,0.0,0.0,null,0.0,500.0,"""500""","""None""",0,"""Donor Country"""
2018,"""DEU""","""Germany""",11,"""ODA Grants""",0.0,0.014163,0.0,0.0,0.01424,0.0,2.0,0.0,0.0,0.0,1.0,0.0,"""Adaptation""","""Adaptation""",1,"""Donor Country"""
2021,"""DEU""","""Germany""",11,"""ODA Grants""",0.00656,0.00035,null,0.006141,0.000327,null,0.0,0.0,1.0,0.0,0.0,500.0,"""500""","""None""",0,"""Donor Country"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018,"""USA""","""United States""",11,"""ODA Grants""",null,0.0,0.05002,null,0.0,0.057689,0.0,0.0,0.0,null,0.0,500.0,"""500""",null,1,"""Donor Country"""
2020,"""USA""","""United States""",11,"""ODA Grants""",null,0.0,null,null,0.0,null,0.0,0.0,0.0,null,0.0,500.0,"""500""","""None""",0,"""Donor Country"""
2005,"""NLD""","""Netherlands""",11,"""ODA Grants""",null,0.224,null,null,0.253721,null,null,null,null,null,0.0,500.0,"""500""",null,1,"""Donor Country"""


In [23]:
from functions.data_operations import fetch_data

fetch_data("donors")

,effective_year,DonorName,climate_class_number,climate_class,meta_category,no_projects,effective_funding,country_code,gdp
0,2000,Australia,0,Adaptation,Adaptation,21,3.201852,AUS,1.343450e+12
1,2000,Australia,1,Solar-energy,Mitigation,2,0.001158,AUS,1.343450e+12
2,2000,Australia,3,Other-mitigation-projects,Mitigation,1,0.008688,AUS,1.343450e+12
3,2000,Australia,4,Biodiversity,Environment,9,0.060237,AUS,1.343450e+12
4,2000,Australia,6,Nature_conservation,Environment,18,1.359398,AUS,1.343450e+12
...,...,...,...,...,...,...,...,...,...
5112,2019,United States,8,Sustainable-land-use,Environment,354,145.885530,USA,1.966310e+13
5113,2019,United States,10,Renewables-multiple,Mitigation,134,77.759460,USA,1.966310e+13
5114,2019,United States,11,Hydro-energy,Mitigation,23,23.371984,USA,1.966310e+13
5115,2019,United States,12,Energy-efficiency,Mitigation,116,42.596624,USA,1.966310e+13


In [13]:
def from_dummies(
    df: pl.DataFrame,
    selected_vars: list[str],
    separator="_",
) -> pl.DataFrame:
    col_exprs = {}

    # Loop through each column to build the expressions
    for col in df.columns:
        name, value = col.rsplit(separator, maxsplit=1)  # Split column name

        # Process only the columns whose base name is in selected_vars
        if name in selected_vars:
            expr = pl.when(pl.col(col) == 1).then(
                pl.lit(value)
            )  # Ensure value is used as a literal
            col_exprs.setdefault(name, []).append(expr)  # Group by base name

    # Select the coalesced expressions for each selected base name
    return df.select(
        [
            pl.coalesce(exprs).alias(name)  # Combine expressions into a single column
            for name, exprs in col_exprs.items()
        ]
    )


# Example usage
data = pl.DataFrame(
    {
        "climate_class_500": [1, 0, 0],
        "climate_class_Adaptation": [0, 1, 0],
        "climate_class_Mitigation": [0, 0, 1],
        "other_class_Option1": [1, 0, 0],
        "other_class_Option2": [0, 1, 0],
    }
)

# Specify which variables to merge
selected_vars = ["climate_class", "other_class"]

reshaped_data = from_dummies(data, selected_vars)
print(reshaped_data)

shape: (3, 2)
┌───────────────┬─────────────┐
│ climate_class ┆ other_class │
│ ---           ┆ ---         │
│ str           ┆ str         │
╞═══════════════╪═════════════╡
│ 500           ┆ Option1     │
│ Adaptation    ┆ Option2     │
│ Mitigation    ┆ null        │
└───────────────┴─────────────┘
